# SQL in Action: Exploring European Clubs

### **About Football [European Clubs] Database:**

European football clubs are professional teams competing in various **domestic** leagues and international tournaments across **Europe**. Some of the most famous leagues include the **Premier League (England), Serie A (Italy), Bundesliga (Germany), La Liga (Spain), and Ligue 1 (France)**. They play a crucial role in the global popularity and significantly influence global football culture, broadcasting rights, merchandising, and player transfers, making them pivotal in the sport's ecosystem.

**Premier League (England) Founded: 1992**
> Widely regarded as the most popular football league globally, known for its competitive balance and high entertainment value.
- **Notable Clubs:** Manchester United, Liverpool, Chelsea, Arsenal, Manchester City.

**Serie A (Italy) Founded: 1898 (as Serie A in 1929)**
> Known for its tactical approach and strong defensive play, often referred to as "Catenaccio."
- **Notable Clubs:** Juventus, AC Milan, Inter Milan, AS Roma, Napoli.

**Bundesliga (Germany) Founded: 1963**
> Known for its passionate fan bases and high attendance figures; many clubs are owned by their supporters (50+1 rule).
- **Notable Clubs:** Bayern Munich, Borussia Dortmund, RB Leipzig, Bayer Leverkusen.

**La Liga (Spain) Founded: 1929**
> Home to one of the fiercest rivalries in football, El Clásico, between FC Barcelona and Real Madrid.
- **Notable Clubs:** FC Barcelona, Real Madrid, Atlético Madrid, Sevilla FC.

**Ligue 1 (France) Founded: 1933**
> Ligue 1 boasts one of the highest percentages of homegrown players among Europe's top leagues, reflecting a strong focus on youth academies.
- **Notable Clubs:** Olympique de Marseille, Lyon, AS Monaco,Paris Saint-Germain 


<img src="https://miro.medium.com/v2/resize:fit:828/format:webp/0*N2NJbUORAzQ8w8T7.jpg" alt="football_clubs_img" width="72"/>

### Problem statement:
**Performance evaluation** of teams and players in European football clubs using **SQL** and to identify top goal scorers and league winners for each season.
Focus on understanding how factors like **home vs. away** conditions and game probabilities influence team success, providing insights for strategic improvements.


### Dataset

This dataset contains football-related data covering the **Top5 leagues** in **Europe** from **2014-2020**. It is structured like a relational database, which makes it easy to work with, regardless of the problem you want so solve.

These are the following tables:
1. `Appearances`: Every appearance a player has made in one of the Top5 leagues 
2. `Games`: All games played in one of the Top5 leagues
3. `Leagues`: Top5 leagues
4. `Players`: Every player who has played in one of the Top5 leagues
5. `Shots`: All shots taken in one of the Top5 leagues
6. `Teams`: All teams who played in one of the Top5 leagues
7. `Teamstats`: Game statistics by team in one of the Top5 leagues over the specified time window

We will look at the fields below 

<img src="https://i.imgur.com/tvYKGfX.png" alt="football_clubs_db_erd" width="720"/>


In this Exploratory Data Analysis (**EDA**) project we will be doing the **following:**
1. **Window** functions for advanced analytics
2. **Indexing** to enhance query performance and efficiency
3. **Ranking** techniques to identify top performers
4. **Functions** for data manipulation
5. **Views** to simplify complex queries

### Data Loading and Overview

In [1]:
import numpy as np  
import pandas as pd
import sqlite3 as sql

# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/football-database/players.csv
/kaggle/input/football-database/leagues.csv
/kaggle/input/football-database/teams.csv
/kaggle/input/football-database/games.csv
/kaggle/input/football-database/shots.csv
/kaggle/input/football-database/teamstats.csv
/kaggle/input/football-database/appearances.csv


In [2]:
# lets load data
players = pd.read_csv('/kaggle/input/football-database/players.csv',
                      encoding_errors='ignore')  # handle error in encoding
leagues = pd.read_csv('/kaggle/input/football-database/leagues.csv')
teams = pd.read_csv('/kaggle/input/football-database/teams.csv')
games = pd.read_csv('/kaggle/input/football-database/games.csv')
shots = pd.read_csv('/kaggle/input/football-database/shots.csv')
teamstats = pd.read_csv('/kaggle/input/football-database/teamstats.csv')
appearances = pd.read_csv('/kaggle/input/football-database/appearances.csv')

In [3]:
dfs = [players, leagues, teams, games, shots, teamstats, appearances]
df_names = ['players', 'leagues', 'teams', 'games', 'shots', 'teamstats', 'appearances']

# Create a summary table
summary = {
    "DataFrame": [],
    "No. of Columns": [],
    "No. of Rows": [],
    "Datatype Aggregation": []
}

for name, df in zip(df_names, dfs):
    summary["DataFrame"].append(name)
    summary["No. of Columns"].append(df.shape[1])
    summary["No. of Rows"].append(df.shape[0])
    summary["Datatype Aggregation"].append(df.dtypes.value_counts().to_dict())

# Convert the summary into a dataframe
summary_df = pd.DataFrame(summary)

# Display the summary dataframe
summary_df

,DataFrame,No. of Columns,No. of Rows,Datatype Aggregation
0,players,2,7659,"{int64: 1, object: 1}"
1,leagues,3,5,"{object: 2, int64: 1}"
2,teams,2,146,"{int64: 1, object: 1}"
3,games,34,12680,"{float64: 24, int64: 9, object: 1}"
4,shots,11,324543,"{float64: 4, object: 4, int64: 3}"
5,teamstats,16,25360,"{int64: 10, object: 3, float64: 3}"
6,appearances,19,356513,"{int64: 14, float64: 4, object: 1}"


**General Overview:**

The dataset consists of a total of **726,906 rows** and **87 columns** across seven tables.

- The `players`, `leagues`, and `teams` tables are relatively small, focusing on primary identifiers and names.
  
- The `games` table has the most columns. However, many of these columns are related to **betting**, which is outside the scope of our analysis. Since we are unfamiliar with the data and its values, we will **remove** these columns.
  
- The `teamstats` table contains team records for each game. As two teams compete in each game, this table has **twice as many records as games**.

- The `shots` and `appearances` tables contain the most records, with **324,543** and **356,513** rows, respectively. The `shots` table records each player who has taken shot during game, while the `appearances` table tracks all players who appeared in each game.

In [4]:
# view data tables
players

,playerID,name
0,560,Sergio Romero
1,557,Matteo Darmian
2,548,Daley Blind
3,628,Chris Smalling
4,1006,Luke Shaw
...,...,...
7654,9512,Valentino Livramento
7655,9740,Jos S
7656,1025,Daniel Bachmann
7657,9745,Dimitris Giannoulis


The `players` table contains `playerID` and corresponding `name`. There are a total of **7,659 rows**, representing that many unique players across 7 seasons (2014-2020).  
- `playerID` is the unique identifier, and `name` is the player's name.


In [19]:
leagues

,leagueID,name,understatNotation
0,1,Premier League,EPL
1,2,Serie A,Serie_A
2,3,Bundesliga,Bundesliga
3,4,La Liga,La_liga
4,5,Ligue 1,Ligue_1


The `leagues` table contains `leagueID`, `name`, and `understatNotation`. We are analyzing the top five leagues as mentioned above. `leagueID` is the unique identifier.

In [5]:
teams

,teamID,name
0,71,Aston Villa
1,72,Everton
2,74,Southampton
3,75,Leicester
4,76,West Bromwich Albion
...,...,...
141,182,Nancy
142,224,Amiens
143,225,Strasbourg
144,235,Nimes


The `teams` table contains `teamID` and `name`. There are a total of **146 rows**, meaning there are 146 teams participating in the top 5 leagues over the span of 7 seasons. `teamID` is the unique identifier.

In [7]:
# games.columns
games = games.iloc[:, 0:13]  # remove betting related fields 

### SQL - Database Connection
The `sql.connect()` function is used to connect to a **SQLite3 database** named `football-database.db`.

This connection allows us to store, manipulate, and query the tables (e.g., players, leagues, teams) in the database.

Once connected, we can perform operations such as inserting the data, querying for analysis, or exporting results.

In [4]:
db = sql.connect('football-database.db')  #SQLite3 connection

In [5]:
players.to_sql('players', db, if_exists='replace', index=False)
leagues.to_sql('leagues', db, if_exists='replace', index=False)
teams.to_sql('teams', db, if_exists='replace', index=False)
games.to_sql('games', db, if_exists='replace', index=False)
shots.to_sql('shots', db, if_exists='replace', index=False)
teamstats.to_sql('teamstats', db, if_exists='replace', index=False)
appearances.to_sql('appearances', db, if_exists='replace', index=False)

# outputs number of rows inserted in each table

356513

- The `to_sql()` method is called on each DataFrame (players, leagues, teams) to write its contents to the corresponding table in the database.

- The first argument specifies the *table name** in the database. e.g: `players` for the players DataFrame

- The `db` parameter is the database connection established earlier.

- The `if_exists='replace'` argument indicates that if the table already exists in the database, it will be replaced with the new data.

- The `index=False` argument ensures that the DataFrame's index is not written as a separate column in the database.

We have stored all seven dataframes into the **football-database** db.

#### Describe Table

We used the `PRAGMA` statement, as supported by **SQLite**, to describe the table, instead of using the standard `DESC` or `DESCRIBE` commands available in other SQL databases.

In [13]:
# example query
q1 = """
PRAGMA table_info(games);
"""

pd.read_sql_query(q1, db)

,cid,name,type,notnull,dflt_value,pk
0,0,gameID,INTEGER,0,None,0
1,1,leagueID,INTEGER,0,None,0
2,2,season,INTEGER,0,None,0
3,3,date,TEXT,0,None,0
4,4,homeTeamID,INTEGER,0,None,0
5,5,awayTeamID,INTEGER,0,None,0
6,6,homeGoals,INTEGER,0,None,0
7,7,awayGoals,INTEGER,0,None,0
8,8,homeProbability,REAL,0,None,0
9,9,drawProbability,REAL,0,None,0


The PRAGMA function provides details such as *field names, data types, whether fields has null values , default values (if any), and whether a field is a primary key*.

In this case, we observe that there are no primary keys defined. Rather than altering the table to add primary keys, we will create indexes directly to improve query performance later.

In [6]:
# example query
q1 = """
SELECT * FROM teamstats
LIMIT 5
"""

pd.read_sql_query(q1, db)

,gameID,teamID,season,date,location,goals,xGoals,shots,shotsOnTarget,deep,ppda,fouls,corners,yellowCards,redCards,result
0,81,89,2015,2015-08-08 15:45:00,h,1,0.627539,9,1,4,13.8261,12,1,2.0,0,W
1,81,82,2015,2015-08-08 15:45:00,a,0,0.674600,9,4,10,8.2188,12,2,3.0,0,L
2,82,73,2015,2015-08-08 18:00:00,h,0,0.876106,11,2,11,6.9000,13,6,3.0,0,L
3,82,71,2015,2015-08-08 18:00:00,a,1,0.782253,7,3,2,11.8462,13,3,4.0,0,W
4,83,72,2015,2015-08-08 18:00:00,h,2,0.604226,10,5,5,6.6500,7,8,1.0,0,D


The `teamstats` table contains detailed **statistics** for **each game**, with two rows per game: one for the home team and another for the away team.
The columns include:
- `gameID` as unique identifier for each game. `teamID` as identifier for team
- `season, date, location` game information
- `goals, xgoals (expected goals), shots, shots on target`, etc for statistics
- `results` is an important field to analyse game results

#### Using SQL `CASE WHEN` to Pivot Tables

The `CASE WHEN` function is used to create categories based on conditions. When the condition is `TRUE`, it is followed by `THEN` to assign a value, and concluded with `END`. Optionally, `ELSE` is used to assign a value for unsatisfied conditions. These conditions form the columns of the table. 

Aggregate functions like `SUM`, `COUNT`, etc., are applied for each condition to calculate values for that column.

Additionally, `GROUP BY` can be used to summarize the data into the required groups.


In [10]:
# example query
q1 = """
SELECT
    COUNT(CASE WHEN leagueID = 1 THEN 1 END) AS 'Premier League',
    COUNT(CASE WHEN leagueID = 2 THEN 1 END) AS 'Serie A',
    COUNT(CASE WHEN leagueID = 3 THEN 1 END) AS 'Bundesliga',
    COUNT(CASE WHEN leagueID = 4 THEN 1 END) AS 'La Liga',
    COUNT(CASE WHEN leagueID = 5 THEN 1 END) AS 'League 1'
FROM games;
"""

# GROUP BY season
pd.read_sql_query(q1, db)

,Premier League,Serie A,Bundesliga,La Liga,League 1
0,2660,2660,2142,2660,2558


There are **7 seasons** in total, with each season comprising **380 matches** across the top five European leagues, where each team plays **38 matches**. However, the **Bundesliga** has **306 matches** per season, as it features only **18 teams** compared to the **20 teams** in the other leagues.

**Insight:**
In **Ligue 1**, two matches were not played: one was canceled due to a **COVID-19** outbreak, and the other, between **SC Bastia** and **Olympique Lyon**, was abandoned after home supporters invaded the pitch twice to attack opposing players.


### Q1) What is the win rate for home and away teams?
We are again using `CASE WHEN` function to create this table. This helps us understand the performance of teams, especially whether playing on home ground boosts their win rate."

we will calculate the number of home and away wins per season and computes their win percentages.

1. **Home Win Rate**:
   - The `CASE WHEN` condition checks for home wins using `location = 'h' for home ` and `result = 'W'` for wins).
   - If the condition is met, it returns `1`, which is counted by `COUNT`.
   - The total home wins is divided by `COUNT(gameID)/2` to account for wins upon number of games played, since the `teamstats` table has two entries per game (one for home and one for away) and multiplied by 100 to convert to a percentage.

2. **Away Win Rate**:
   - Similarly, `CASE WHEN` is used to check for away wins (`location = 'a'` and `result = 'W'`), and these are counted.
   - The away win percentage is calculated similarly by dividing away wins by the total games.
   
The results are grouped by `season` to get statistics for each season.

In [27]:
# count "gameID/2" give home and away games number
# has each game is played at any one of the two teams' home ground
q1 = """
SELECT season,
    COUNT(CASE WHEN location = 'h' AND result = 'W' THEN 1 END) AS home_wins,
    COUNT(CASE WHEN location = 'h' AND result = 'W' THEN 1 END) * 100 / (COUNT(gameID)/2) as 'home_wins%',  
    COUNT(CASE WHEN location = 'a' AND result = 'W' THEN 1 END) AS away_wins,
    COUNT(CASE WHEN location = 'a' AND result = 'W' THEN 1 END) * 100 / (COUNT(gameID)/2) 'away_wins%'
FROM teamstats
GROUP BY season
"""

results = pd.read_sql_query(q1, db)
results

,season,home_wins,home_wins%,away_wins,away_wins%
0,2014,821,44,531,29
1,2015,810,44,543,29
2,2016,889,48,515,28
3,2017,828,45,551,30
4,2018,817,44,537,29
5,2019,761,44,544,31
6,2020,728,39,633,34


We can look at the **percentage of wins** for both home and away teams.
From the data, it is clear that **home team wins** are consistently **greater** than away team wins. This suggests that home team **support** plays a significant role, as well the players being more familiar with the pitch conditions.

There appears to be a **decreasing trend** in the win percentage for home teams over the years, Simultaneously, there is a clear increase in away team wins.

The home win rate increased until 2016 and then decreased by the end of 2020. There was 48% chance nearly 50% of the home team winning the game in 2016 season which is quite high. Simultaneously the win rate for away teams was 28% 

The win rate for home dropped to **39%** in 2020. This could indicate a diminishing advantage for home teams, possibly due to factors like improved away team preparation or changing conditions, such as fewer fans in stadiums (e.g., due to COVID-19 restrictions in 2020).
The win rate for away team is nearly equal 34%. 

### Q2) What is the average number of goals scored to win games?
Unlike teamstats, games table can be quickly used to asses goals scored as both team values are present hanldly in each row

#### Common Table Expression (CTE) in SQL
We'll use a CTE to categorize the outcomes of games. It is a **temporary result**. It is created using the `WITH` keyword and can simplify complex queries by breaking them into more manageable parts.

To calculate the average number of goal scored to win the game by both home and away teams we use CASE WHEN to classify the games. The game_outcomes CTE categorizes each game as a **"Home Win", "Away Win" or "Draw"** based on the goals scored.

The main query first uses above CTE `'game_outcomes'`, the data is then **grouped by** 'season' and 'result' to provide summary statistics for each outcome type. It calculates the **count of games** for each result as `'count'`, the **average home goals** as `'avg_home_goals'`, and the **average away goals** as `'avg_away_goals'`, giving insights into scoring trends by season and result type.


In [31]:
q1 = """
WITH game_outcomes AS (
    SELECT 
        gameID,
        season,
        CASE 
            WHEN homeGoals > awayGoals THEN 'Home Win'
            WHEN homeGoals < awayGoals THEN 'Away Win'
            ELSE 'Draw'
        END AS result,
        homeGoals,
        awayGoals
    FROM games
)

SELECT 
    season,
    result,
    COUNT(*) AS count,
    AVG(homeGoals) AS avg_home_goals,
    AVG(awayGoals) AS avg_away_goals
FROM game_outcomes
GROUP BY season, result;
"""

results = pd.read_sql_query(q1, db)
results.set_index('season')

,result,count,avg_home_goals,avg_away_goals
season,,,,
2014,Away Win,531,0.580038,2.220339
2014,Draw,474,0.955696,0.955696
2014,Home Win,821,2.405603,0.523752
2015,Away Win,543,0.565378,2.268877
2015,Draw,473,0.972516,0.972516
2015,Home Win,810,2.467901,0.513580
2016,Away Win,515,0.603883,2.456311
2016,Draw,421,0.976247,0.976247
2016,Home Win,889,2.525309,0.596175


To **win** a game, both **home** and **away** teams generally need to score around **2.4 goals** on average while limiting the opposing team with strong defence to about **0.55 goals** per match.

For **draws**, both teams tend to score close to **1 goal** each. This balance in scoring suggests that when teams are evenly matched, games are more likely to end without a winner. 

Both the stats are consistent same across all the seasons 


In [33]:
# Segregating results for further analysis
results1 = results[results['result'] == 'Away Win']
results2 = results[results['result'] == 'Home Win']
results1.set_index('season')
results2.set_index('season')

,result,count,avg_home_goals,avg_away_goals
season,,,,
2014,Home Win,821,2.405603,0.523752
2015,Home Win,810,2.467901,0.513580
2016,Home Win,889,2.525309,0.596175
2017,Home Win,828,2.458937,0.519324
2018,Home Win,817,2.451652,0.545900
2019,Home Win,761,2.462549,0.536137
2020,Home Win,728,2.532967,0.631868


- Additionally we use pandas' functionality to **filter** the data for further analysis.
- We got a better view at Home wins data output.
- SQL isn't always required, **Pandas** can often handle **data manipulations** more efficiently.

The filtered data for **Home Wins** from 2014 to 2020 shows a consistent trend: **home teams** typically score around **2.4 to 2.5 goals** per game, with their opponents limited to about **0.5 goals** on average.

Over these seven seasons, **home teams** maintained strong offensive performances, peaking at an average of **2.53 goals** in 2020.

Defensive consistency is also evident, with **away teams** scoring only between **0.51 and 0.63 goals**. This pattern highlights the home advantage, where effective scoring and robust defense play critical roles in securing wins.

The **'count'** column reveals the total number of **Home Wins** each season, showing a slight decrease over the years as seen earlier. 

### Index Creation for Database Tables

Indexing is a crucial optimization technique that enhances the speed of data retrieval operations on a database table. Indexes serve as lookup tables to efficiently store data, often sorting it internally, which allows the database to access the relevant rows more efficiently and quickly during query execution.
 
The tables and fields selected for indexing in this code are: 
- `players` table on the `playerID` field
- `leagues` table on the `leagueID` field
- `teams` table on the `teamID` field
- `games` table on the `gameID` field

I chose to index only these four fields as they have unique entries, which allows for efficient and fast access to the data.

To test the optimization done by indexing, before creation of index we will execute a query which will be used to get average goals again. We will use magic function `%%time` to record the total time taken for execution of query.

To test **indexing optimization**, we’ll first execute a query to get average goals before creating the index. Using the `%%time` magic function, we’ll record the query's execution time. We will use this query again after creating an index. allowing us to compare performance and observe any improvements. 

In [19]:
%%time

# execution time before index creation
q1 = """
SELECT 
    g.leagueID,
    l.name AS league_name,
    AVG(g.homeGoals + g.awayGoals) AS avg_total_goals
FROM 
    games g
JOIN 
    leagues l ON g.leagueID = l.leagueID
GROUP BY 
    g.leagueID, l.name;
"""

pd.read_sql_query(q1, db)

CPU times: user 18.5 ms, sys: 2.07 ms, total: 20.6 ms
Wall time: 18.3 ms


,leagueID,league_name,avg_total_goals
0,1,Premier League,2.711654
1,2,Serie A,2.810526
2,3,Bundesliga,2.952381
3,4,La Liga,2.658647
4,5,Ligue 1,2.602033


In [6]:
create_index_queries = [
    "CREATE UNIQUE INDEX idx_playerID ON players(playerID);",

    "CREATE UNIQUE INDEX idx_leagueID ON leagues(leagueID);",

    "CREATE UNIQUE INDEX idx_teamID ON teams(teamID);",

    "CREATE UNIQUE INDEX idx_gameID ON games(gameID);",
]

for query in create_index_queries:
#     print(query)
    db.execute(query)  # Use db.execute() for SQL commands that don't return results
    db.commit()  # Ensure the changes are committed

`pd.read_sql_query()` is designed to retrieve result sets from the database. Since the queries for creating indexes return no results, they produce a `NoneType` object, which can led to errors. 

Instead `db.execute()` function is utilized to execute the SQL statements that create the indexes. After executing the query, the changes are committed to the database to ensure they take effect usin db.commit()

### Q3) What are the average goals scored in each league?

We use `%%time` to record the query's execution time, as discussed above to measure the performance and assess any improvements after optimization using index.

The selecd statement uses expression `AVG(g.homeGoals + g.awayGoals)` to compute the average **total goals** scored per game by adding both home and away goals. This provides a view on scoring trends for each game across each league.

The query uses an **inner join** to combine data from the `games` table (`g`) and the `leagues` table (`l`). It matches records based on the common column `leagueID` in both tables, allowing us to include the league's name in the results.

The query groups the data by `g.leagueID` and `l.name` so that the **average total goals** to get results for each league, providing a distinct `avg_total_goals` for each.


In [35]:
%%time

# execution time after index creation
q1 = """
SELECT 
    g.leagueID,
    l.name AS league_name,
    AVG(g.homeGoals + g.awayGoals) AS avg_total_goals
FROM 
    games g
JOIN 
    leagues l ON g.leagueID = l.leagueID
GROUP BY 
    g.leagueID, l.name;
"""

pd.read_sql_query(q1, db)

CPU times: user 17.4 ms, sys: 3.03 ms, total: 20.4 ms
Wall time: 17.7 ms


,leagueID,league_name,avg_total_goals
0,1,Premier League,2.711654
1,2,Serie A,2.810526
2,3,Bundesliga,2.952381
3,4,La Liga,2.658647
4,5,Ligue 1,2.602033


**Indexing**
- After creating indexes on the `games` and `leagues` tables, we noticed only a minor reduction in query execution time, approximately 1 millisecond. 
- Since the games table has a moderate size and the queries executed are relatively short.
- This slight improvement indicates that indexing remains beneficial, providing faster data retrieval even when the differences are minimal.


**Results**

Among the top 5 leagues in Europe, the Bundesliga boasts the **highest** average of  **2.95 goals per game**, nearly 3 goals per match. This high-scoring environment is characteristic of the **Bundesliga**, known for its attacking style of play and dynamic football culture. Clubs like **Bayern Munich** and **Borussia Dortmund** consistently showcase offensive prowess.

In contrast, **Ligue 1** has the **lowest** average, with about **2.60 goals per game**. While this may suggest a more defensive approach. The average remains lower due to tactical approaches and sometimes conservative gameplay. Ligue 1 features strong defensive teams, such as **Paris Saint-Germain and AS Monaco**.

In [19]:
# # get the higest and lowest
# q1 = """
# SELECT 
#     strftime('%Y-%m', ts.date) AS month_year,
#     SUM(ts.yellowCards) AS total_yellow_cards,
#     SUM(ts.redCards) AS total_red_cards
# FROM 
#     teamstats ts
# GROUP BY 
#     month_year
# ORDER BY 
#     month_year;
# """

# pd.read_sql_query(q1, db)


,month_year,total_yellow_cards,total_red_cards
0,2014-08,412.0,20
1,2014-09,754.0,33
2,2014-10,681.0,36
3,2014-11,780.0,41
4,2014-12,820.0,45
...,...,...,...
66,2021-01,1000.0,31
67,2021-02,833.0,32
68,2021-03,574.0,14
69,2021-04,854.0,44


### Q4) Who are the players with the highest goals and assists? 
Retrieve data on players with the highest number of goals and assists to identify top performers in terms of overall contributions to their team.


#### Ranking function
Ranking functions in SQL assign a unique rank to each row within a partition of a result set, based on specified criteria. They facilitate ordered results by allowing for ranking methods like `ROW_NUMBER()`, `RANK()`, and `DENSE_RANK()`, which identifies the position of each record in relation to others. This is useful for tasks like identifying top performers or ranking items within a category.

WITH `PlayerGoals` as CTE, it computes the total number of goals for each player by joining `appearances` and `players` table.

SUM(a.goals) sums up the goals for each player from the appearances table.
The JOIN operation connects the appearances table with the players table to fetch the player's name.

RANK() OVER (ORDER BY total_goals DESC) on `PlayerGoals` CTE, ranks players based on their **total goals** in descending order. The player with the highest total gets **rank** 1. We limit the outcome to top 11.

In [38]:
q1 = """
WITH PlayerGoals AS (
    SELECT 
        a.playerID,
        pl.name,
        SUM(a.goals) AS total_goals
    FROM 
        appearances a
    JOIN 
        players pl ON a.playerID = pl.playerID
    GROUP BY 
        a.playerID, pl.name
)
SELECT 
    playerID,
    name,
    total_goals,
    RANK() OVER (ORDER BY total_goals DESC) AS player_rank
FROM PlayerGoals
LIMIT 11;
"""

pd.read_sql_query(q1, db)

,playerID,name,total_goals,player_rank
0,2097,Lionel Messi,231,1
1,2371,Cristiano Ronaldo,215,2
2,227,Robert Lewandowski,203,3
3,2098,Luis Surez,168,4
4,647,Harry Kane,163,5
5,318,Pierre-Emerick Aubameyang,149,6
6,1209,Ciro Immobile,133,7
7,619,Sergio Agero,132,8
8,3294,Edinson Cavani,132,8
9,1250,Mohamed Salah,130,10


The query results display the **top 11 goal scorers** from the **2014 to 2020** seasons, highlighting their impressive goal-scoring feats. 

**Lionel Messi** leads with **231 goals**, securing the **1st** rank, followed closely by **Cristiano Ronaldo** with **215 goals** in **2nd** place. **Robert Lewandowski** rounds out the top three with **203 goals**, showcasing his remarkable striking ability.

This data underscores the consistency and scoring prowess of these elite players over the analyzed seasons, making them some of the most impactful forwards in modern football history.

Similarly, to get the player highest number of assists. We create CTE as `PlayerAssits` using same table for counting assits by **each player** to get **total assists**

In [39]:
q1 = """
WITH PlayerAssits AS (
    SELECT 
        a.playerID,
        pl.name,
        SUM(a.assists) AS total_assists
    FROM 
        appearances a
    JOIN 
        players pl ON a.playerID = pl.playerID
    GROUP BY 
        a.playerID, pl.name
)
SELECT 
    playerID,
    name,
    total_assists,
    RANK() OVER (ORDER BY total_assists DESC) AS player_rank
FROM PlayerAssits
LIMIT 11;
"""

pd.read_sql_query(q1, db)
# team and league name

,playerID,name,total_assists,player_rank
0,447,Kevin De Bruyne,97,1
1,2097,Lionel Messi,97,1
2,224,Thomas Mller,87,3
3,3553,ngel Di Mara,75,4
4,2098,Luis Surez,72,5
5,536,Dimitri Payet,70,6
6,2099,Neymar,61,7
7,617,David Silva,59,8
8,1491,Alejandro Gomez,59,8
9,646,Christian Eriksen,56,10


The query results display the **top 11 players** based on assists from the **2014 to 2020** seasons, highlighting their contributions to goal creation. 

**Kevin De Bruyne** and **Lionel Messi** are tied for the **1st rank**, both accumulating **97 assists** during this period, showcasing their exceptional playmaking abilities. Following them is **Thomas Müller** in **3rd place** with **87 assists**, demonstrating his skill in creating scoring opportunities.  

This results shows us  **Lionel Messi's** remarkable versatility, as he tops both the goal-scoring and assist charts, further solidifying his status as one of the greatest players in football history.


### Q5) Who are the league winners in each edition?
We have created `top_5_teams_per_league` view in which we have `team_wins` CTE using team ID, season, league ID, count of result from `teamstats` and `games` table using inner join.

This subquery calculates the number of wins (COUNT(all)) for each team in each season and league, joining `teamstats` with `games` on gameID and filtering for result = 'W'.

This `ranked_teams` CTE ranks teams within each league and season using `ROW_NUMBER()` based on the number of wins in `descending` order. We **partition** by season and leagueID to rank teams within each `league` and `season`.

The main query selects the **top 5 teams** per season and league by filtering WHERE ranked_teams `rank <= 5`.
It joins the teams and leagues tables to get the respective team and league names using the name column.

#### Views

Views versus Common Table Expressions (CTE)
- A view is a persistent object stored in the database. Once created, it remains available for use until explicitly dropped. You can think of a view as a virtual table.
- A view is defined with a CREATE VIEW statement and can be updated or dropped as needed.

- A CTE is a temporary result set that exists only for the duration of a single query. Once the query execution is completed, the CTE ceases to exist.
- A CTE is defined using the WITH clause at the beginning of a query. You can think of it as a named temporary result set.

In [8]:
# the teams with the highest wins in a season is league winner
q1 = """
CREATE VIEW top_5_teams_per_league AS
WITH team_wins AS (
    SELECT ts.teamID, g.season, g.leagueID, COUNT(ts.result) AS wins
    FROM teamstats ts
    JOIN games g ON ts.gameID = g.gameID
    WHERE ts.result = 'W'
    GROUP BY ts.teamID, g.season, g.leagueID
),
ranked_teams AS (
    SELECT 
        tw.teamID, 
        tw.season, 
        tw.leagueID, 
        tw.wins,
        ROW_NUMBER() OVER (PARTITION BY tw.season, tw.leagueID ORDER BY tw.wins DESC) AS rank
    FROM team_wins tw
)
SELECT 
    rt.season, 
    l.name AS league, 
    t.name AS team, 
    rt.wins,
    rt.rank
FROM ranked_teams rt
JOIN teams t ON rt.teamID = t.teamID
JOIN leagues l ON rt.leagueID = l.leagueID
WHERE rt.rank <= 5
ORDER BY rt.season, l.name, rt.rank;
"""

db.execute(q1)

In [9]:
q1 = """
SELECT * FROM top_5_teams_per_league 
WHERE season = 2018 and rank = 1;
"""

pd.read_sql_query(q1, db)

,season,league,team,wins,rank
0,2018,Bundesliga,Bayern Munich,24,1
1,2018,La Liga,Barcelona,26,1
2,2018,Ligue 1,Paris Saint Germain,29,1
3,2018,Premier League,Manchester City,32,1
4,2018,Serie A,Juventus,28,1


The query retrieves the **top ranked teams** for the **2018 season** across various leagues by filtering the `'top_5_teams_per_league'` table for teams that ranked 1.  

The results showcase the dominance of these teams in their respective leagues:

- In the **Bundesliga**, **Bayern Munich** secured the top spot with **24 wins**.
- **Barcelona** led **La Liga** with an impressive **26 wins**.
- **Paris Saint-Germain** (PSG) topped **Ligue 1** with a remarkable **29 wins**.
- **Manchester City** emerged as the leaders in the **Premier League** with a record **32 wins**.
- **Juventus** dominated **Serie A** with **28 wins**, showcasing their consistent performance.

This query highlights the successful campaigns of these clubs, reflecting their strengths and strategies during the 2018 season, as they emerged as champions in their respective leagues.

# El classico
El Clásico is one of the most **iconic rivalries** in **football**, featuring fierce competition between **Barcelona** and **Real Madrid**. The matches are not only a showcase of **elite talent** but also a clash of two **distinct cultures and philosophies**, captivating fans worldwide.

#### LIKE Operator
The **LIKE** operator in SQL is used to search for **patterns** within text fields, making it possible to find partial matches by using **wildcards**: `%` matches any **sequence** of characters and `_` matches a **single** character.

One disadvantage of using LIKE when the exact value is known is its reduced efficiency compared to **equal to (`=`)**. Since LIKE performs pattern matching, requiring more processing, which can slow down queries, particularly in large datasets where exact matches would be more efficient.

In [32]:
# search using like, assuming we 
q1 = """
SELECT * FROM teams 
WHERE name LIKE 'Bar%' OR name LIKE "%Real M%";
"""

pd.read_sql_query(q1, db)

,teamID,name
0,148,Barcelona
1,150,Real Madrid


In [7]:
# let us again create a view
q1 = """
CREATE VIEW el_classico AS
SELECT 
    ts.*,
    t.name,
    g.homeProbability, 
    g.drawProbability, 
    g.awayProbability
FROM teamstats ts
NATURAL JOIN teams t 
JOIN games g ON ts.gameID = g.gameID
WHERE (g.homeTeamID = 148 AND g.awayTeamID = 150)
   OR (g.homeTeamID = 150 AND g.awayTeamID = 148);
"""    
db.execute(q1)

In [11]:
q1 = """
SELECT * FROM el_classico;
"""

pd.read_sql_query(q1,db)

,gameID,teamID,season,date,location,goals,xGoals,shots,shotsOnTarget,deep,ppda,fouls,corners,yellowCards,redCards,result,name,homeProbability,drawProbability,awayProbability
0,1510,150,2015,2015-11-21 21:15:00,h,0,1.258100,13,7,5,12.1563,13,10,3.0,1,L,Real Madrid,0.0685,0.1225,0.8090
1,1510,148,2015,2015-11-21 21:15:00,a,4,3.276920,18,7,18,9.8182,10,4,2.0,0,W,Barcelona,0.0685,0.1225,0.8090
2,1702,148,2015,2016-04-02 22:30:00,h,1,0.882445,14,3,14,5.1389,17,4,3.0,0,L,Barcelona,0.1854,0.2562,0.5584
3,1702,150,2015,2016-04-02 22:30:00,a,2,1.565370,15,6,8,15.8846,15,3,2.0,1,W,Real Madrid,0.1854,0.2562,0.5584
4,3164,148,2016,2016-12-03 19:15:00,h,1,1.739840,11,3,7,5.4545,19,5,4.0,0,D,Barcelona,0.4790,0.2541,0.2669
5,3164,150,2016,2016-12-03 19:15:00,a,1,1.359090,14,5,3,10.8214,16,7,2.0,0,D,Real Madrid,0.4790,0.2541,0.2669
6,4047,150,2016,2017-04-23 19:45:00,h,2,2.899740,22,14,15,10.5667,13,7,2.0,1,L,Real Madrid,0.5760,0.2037,0.2203
7,4047,148,2016,2017-04-23 19:45:00,a,3,2.060860,16,9,8,8.0833,8,4,2.0,0,W,Barcelona,0.5760,0.2037,0.2203
8,5600,148,2014,2015-03-22 23:00:00,h,2,2.296120,17,5,5,10.0400,9,7,5.0,0,W,Barcelona,0.5169,0.2336,0.2495
9,5600,150,2014,2015-03-22 23:00:00,a,1,1.699070,13,4,9,8.3714,14,10,6.0,0,L,Real Madrid,0.5169,0.2336,0.2495


lets get some interesting insights
The `el_classico` view contains detailed statistics for each of the **14 matches** played between **Barcelona** and **Real Madrid** from **2014 to 2020**. It includes information on:

**Team Performance Metrics:** Goals, expected goals (xGoals), shots, and shots on target.
**Game Context:** Each match's location (home or away), date, and season.
**Aggressive Play Indicators:** Fouls, corners, yellow and red cards.
**Win Probability:** Home, draw, and away probabilities for each game.

### Q6) What is the total number of El Clasico wins for Barcelona and Real Madrid?

#### User defined Functions in SQL
User-defined Functions in SQL  routines that perform specific operations on data, returning a single value or a table. 

We have used a function to pass the team ID to get the respective win counts
We can pass the team ID and the "?" question mark in the where clause is replaced with the team ID to perform the following query.

In [33]:
def get_team_wins(team_id):
    q = """
    SELECT 
        name, 
        SUM(CASE WHEN result = 'W' THEN 1 ELSE 0 END) AS win_count
    FROM el_classico
    WHERE teamID = ?
    GROUP BY name;
    """
    return pd.read_sql_query(q, db, params=(team_id,))

In [34]:
# Get win count for team with ID 148
get_team_wins(148)

,name,win_count
0,Barcelona,6


In [31]:
get_team_wins(150)

,name,win_count
0,Real Madrid,5


Between Barcelona and Real Madrid, Barcelona has **won 6 times** that is more than that of Real Madrid who has **won five times** out of 14 El Classico matches the played in season 2014 - 2020.

### Q7) Games won against win probability? 
Wins achieved against the odds, where teams won despite lower win probabilities.

If a team played at **home (`location = 'h'`)** and **won (`result = 'W'`)**, when their **home probability** was lower than **away probability**(`awayProbability > homeProbability`). This means the team won despite being less likely to win at home.

Similarly, if a team played **away (`location = 'a'`)** and **won**, when their **home probability** was greater than the **away probability** (`homeProbability > awayProbability`). This implies an away win against expectations.

This query highlights surprising victories in **El Clásico**, where either **Real Madrid** or **Barcelona** won while considered the underdog, providing a unique look at games that defied the odds.

In [36]:
q1 = """
SELECT *
FROM el_classico
WHERE 
    (location = 'h' AND result = 'W' AND awayProbability > homeProbability)
    OR
    (location = 'a' AND result = 'W' AND homeProbability > awayProbability);
"""

pd.read_sql_query(q1,db)

,gameID,teamID,season,date,location,goals,xGoals,shots,shotsOnTarget,deep,ppda,fouls,corners,yellowCards,redCards,result,name,homeProbability,drawProbability,awayProbability
0,4047,148,2016,2017-04-23 19:45:00,a,3,2.06086,16,9,8,8.0833,8,4,2.0,0,W,Barcelona,0.5760,0.2037,0.2203
1,12279,150,2019,2020-03-01 20:00:00,h,2,1.06170,13,5,14,14.0741,13,8,2.0,0,W,Real Madrid,0.2484,0.2729,0.4787


There have been two occasions in **El Clasico** where a team won against the statistical **odds** of winning, defying pre-match win probabilities.

**Barcelona’s Underdog Victory (2016)**: On April 23, 2017, Barcelona claimed a memorable **3-2 away victory** against Real Madrid despite having a 22% win probability. They dominated with 16 shots and 9 on target, showcasing exceptional offensive resilience despite the low odds.

**Madrid’s Triumph (2019)**: On March 1, 2020, Real Madrid secured a **2-0 home win** over Barcelona with just a 24% win probability. Despite being less favored, Real Madrid capitalized on their opportunities, showing strong defense to counter Barcelona’s threat and ultimately winning against odds.

These two matches stand out as significant moments in the rivalry, where each team outperformed statistical projections to secure dramatic wins.

### Q8) What are the results and stats for each match?
We will create a **consolidated view** for **each game** using **concat and join** function. Here we can directly view the home team and away team the goal they scored against each other i.e. scoreboard. The shots taken, the penalties made and the probability of win and lose, results etc.

In [11]:
q1 = """
SELECT 
    g1.gameID, 
    DATE(g1.date) as date,
    g1.name AS home_team,
    g2.name AS away_team,
    CONCAT(g1.goals, ':', g2.goals) AS goals,
    CONCAT(g1.shots, ':', g2.shots) AS shots,
    CONCAT(g1.shotsOnTarget, ':', g2.shotsOnTarget) AS shots_on_target,
    CONCAT(g1.yellowCards, ':', g2.yellowCards) AS yellow_cards,
    CONCAT(g1.redCards, ':', g2.redCards) AS red_cards,
    CONCAT(g1.corners, ':', g2.corners) AS corners,
    CONCAT(g1.fouls, ':', g2.fouls) AS fouls,
    CONCAT(g1.deep, ':', g2.deep) AS deep,
    CONCAT(g1.ppda, ':', g2.ppda) AS ppda,
    g1.result AS home_result,
    g2.result AS away_result,
    g1.homeProbability,
    g2.awayProbability
FROM el_classico g1
JOIN el_classico g2 ON g1.gameID = g2.gameID
WHERE g1.location = 'h' AND g2.location = 'a';
"""

results = pd.read_sql_query(q1,db)
results

,gameID,date,home_team,away_team,goals,shots,shots_on_target,yellow_cards,red_cards,corners,fouls,deep,ppda,home_result,away_result,homeProbability,awayProbability
0,1510,2015-11-21,Real Madrid,Barcelona,0:4,13:18,7:7,3.0:2.0,1:0,10:4,13:10,5:18,12.1563:9.8182,L,W,0.0685,0.8090
1,1702,2016-04-02,Barcelona,Real Madrid,1:2,14:15,3:6,3.0:2.0,0:1,4:3,17:15,14:8,5.1389:15.8846,L,W,0.1854,0.5584
2,3164,2016-12-03,Barcelona,Real Madrid,1:1,11:14,3:5,4.0:2.0,0:0,5:7,19:16,7:3,5.4545:10.8214,D,D,0.4790,0.2669
3,4047,2017-04-23,Real Madrid,Barcelona,2:3,22:16,14:9,2.0:2.0,1:0,7:4,13:8,15:8,10.5667:8.0833,L,W,0.5760,0.2203
4,5600,2015-03-22,Barcelona,Real Madrid,2:1,17:13,5:4,5.0:6.0,0:0,7:10,9:14,5:9,10.04:8.3714,W,L,0.5169,0.2495
5,5776,2014-10-25,Real Madrid,Barcelona,3:1,18:15,6:5,2.0:4.0,0:0,3:9,11:10,8:7,12.6667:7.6176,W,L,0.5211,0.2182
6,8046,2017-12-23,Real Madrid,Barcelona,0:3,14:18,5:11,2.0:2.0,1:0,7:7,14:10,7:13,12.3333:13.7,L,W,0.0464,0.8643
7,8236,2018-05-06,Barcelona,Real Madrid,2:2,11:17,4:5,3.0:5.0,1:0,6:5,8:20,6:13,16.8824:7.1081,D,D,0.2648,0.4691
8,10047,2018-10-28,Barcelona,Real Madrid,5:1,13:15,8:4,2.0:2.0,0:0,4:2,17:14,11:7,11.7692:12.2222,W,L,0.6222,0.1813
9,10207,2019-03-02,Real Madrid,Barcelona,0:1,17:9,3:4,3.0:2.0,0:0,8:6,16:13,8:7,10.5385:12.7368,L,W,0.2161,0.4835


**Game 4047 (2017)**: As seen earlier Barcelona had won an away game despite the odds. Here, Real Madrid created 22 chances more then opposition and more than in any match, with 14 shots on target causing significant offensive pressure, but Barcelona’s defense and goalkeeping turned the tide, securing an unexpected away victory.

**Game 3164 (2016)**: This game ended in a **1-1 draw** with Barcelona at home. It was notable for its **high foul** count, with both teams combining for 19 fouls. This intense physicality, likely part of each team’s defensive strategy, underscored the **fierce competition** typical of El Clásico.

**Games with Dominant Away Team Odds:**

**Game 1510 (2015)**: Barcelona’s away win with a **81% win probability** confirms the strength of their squad that season, clinching a commanding **4-0** victory over Real Madrid.

**Game 8046 (2017)**: Barcelona again defeated Real Madrid with a high **86% away win probability**, and the game ended **3-0** in their favor. 

**Game 15074 (2021)**: In this match, Real Madrid as the away team took a **3-1 victory** over Barcelona with a **75% win probability** before match. This reversal of the usual El Clásico tension demonstrates Real Madrid’s strong form leading into the game.

More such findings can be viewed from the above consolidated result table.

### Summary
This EDA project analyzed a comprehensive football dataset, consisting of **726,906 rows** and **87 columns** across seven tables in an **SQLite database**. The dataset was connected using `sql.connect()` and populated with tables like players, leagues, and teams using `to_sql()`. Through this structured analysis, several insights emerged:

- **Home Advantage**: Home teams consistently achieved higher win rates than away teams, suggesting significant support impact.
- **Scoring Trends**: To secure a win, both home and away teams averaged around **2.4 goals**, while maintaining strong defense, limiting opponents to an average of **0.55 goals**. Draws saw each team scoring around 1 goal.
- **League Comparisons**: **Bundesliga** averaged the **highest** scoring rate at **2.95 goals per game**, while **Ligue 1** had the lowest at **2.6 goals**.
- **Top Performers**: From **2014 to 2020**, **Lionel Messi** led with **231 goals** and ranked first in assists **97**, **sharing** this position with **Kevin De Bruyne**.
- **League Champions**: In **2018**, top teams across leagues included `Bayern Munich (Bundesliga), Barcelona (La Liga), PSG (Ligue 1), Manchester City (Premier League), and Juventus (Serie A)`.
- **El Clásico Highlights**: Between **2014 and 2020**, **Barcelona** won **6** times in **El Clásico**, edging out **Real Madrid’s 5** wins. Notable matches included surprising upsets, like **Barcelona’s 3-2 away victory** with a **22% win probability** in 2017, and **Real Madrid’s 2-0 home win** in 2020 with only a **24% win probability**.

### Conclusion:

This SQL project explored key aspects of the dataset through `window functions, ranking techniques, SQL functions, views, and indexing`. Key questions were addressed, including win rates for home and away teams, average goals for wins, league scoring trends, and player performance metrics like goals and assists. The analysis also highlighted team dominance in various leagues and historic El Clásico outcomes, offering valuable insights into performance dynamics, scoring, and match outcomes across seasons. The project effectively demonstrates the utility of `SQL` for detailed sports data `analysis`.

## References
- [Kaggle notebook for quick analysis on data](https://www.kaggle.com/code/technika148/diving-into-the-world-of-football)
- [SQL basic Cheatsheet](https://www.datacamp.com/cheat-sheet/sql-basics-cheat-sheet)
- [Comprehensive Cheatsheet on QuickRef.Me](https://quickref.me/mysql.html)

## Future Work
- Draw more insightes on players like shoot accuracy, goal score percentage, passes, etc
- Insights on each league(rework the same format) (number of players, top 5 of each league, journey of season winners through out the campaign with timeline)